In [ ]:
import psycopg2
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import Series

import matplotlib.pyplot as plt
import numpy as np
import itertools

import numpy
from numpy import concatenate
from numpy import array

import seaborn as sns

import math
from math import sqrt

# ARIMA AND LSTM STUFF
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tools.eval_measures import rmse

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

import time
import statsmodels.api as sm
import warnings

# Get data from database tables 

In [ ]:
'''from google.colab import drive
drive.mount("/content/drive")
path = '/content/drive/My Drive/DS4A-MX7/config_anahuac_copy.json'

import json
with open(path) as f:
    config_file = json.load(f)
    
# connecting to external database
pd.options.display.max_columns = 500
connection = psycopg2.connect(user = config_file['user'],
                            password = config_file['password'],
                            host= config_file['host'],
                            port = "5432",
                            database = config_file['dbname'])

'''

connection = psycopg2.connect(user = "equipo7",
                            password = "DLgndXy2m4hbWH-qn-Co",
                            host= "ds4a-demo-instance.cssn41frspmj.us-east-1.rds.amazonaws.com",
                            port = "5432",
                            database = "anahuac")



cursor = connection.cursor()
def runquery(query):
    df=pd.read_sql(query,connection)
    return df

In [ ]:
def pivot_by_factor(df, index_table, column_table, value_table):
    df = pd.pivot_table(df, index = index_table, columns = column_table, values = value_table)
    df = df.sort_index(axis=1, level=1)
    df.columns = [f'{b}_{a}' for a, b in df.columns]
    df = df.reset_index()
    return df

In [ ]:
where_date_clause = " WHERE date >= '2012-01-01' AND date <= '2020-03-01 00:00:00' ORDER BY date"; 

sqlQuery_radiacion = "select * from processed_metrics.radiacion " + where_date_clause
df_radiacion = pd.DataFrame(runquery(sqlQuery_radiacion))

sqlQuery_rama = "select * from processed_metrics.rama " + where_date_clause
df_rama = pd.DataFrame(runquery(sqlQuery_rama))

sqlQuery_redmet = "select * from processed_metrics.redmet " + where_date_clause
df_redmet = pd.DataFrame(runquery(sqlQuery_redmet))

sqlQuery_presion = "select * from processed_metrics.presion " + where_date_clause
df_presion = pd.DataFrame(runquery(sqlQuery_presion))

sqlQuery_weather = "select date,fecha,hora,dewp_c AS dew_point_st,rh_st AS rain_st from processed_metrics.weather_fix  " + where_date_clause
df_weather = pd.DataFrame(runquery(sqlQuery_weather))

In [ ]:
#radiation only has data until 2019-12-31 23:00:00. pressure has one more month but we will not consider it
#process radiation
df_radiacion = pivot_by_factor(df_radiacion,["date","fecha","hora"],["factor"],["st"])

#process pa
df_presion.rename(columns={'st': 'PA_st'}, inplace=True)
del df_presion['factor']

#process pollutants
df_rama = pivot_by_factor(df_rama,["date","fecha","hora"],["factor"],["st0","st1","st2"])

#process redmet
df_redmet = pivot_by_factor(df_redmet,["date","fecha","hora"],["factor"],["st0","st1","st2"])

#weather, we are interested in rain and dewpoint, already cleaned when reading from DB


In [ ]:
#merge all data as one
from functools import reduce
dfs = [df_rama, df_redmet, df_presion, df_radiacion, df_weather]
df_workspace = reduce(lambda left,right: pd.merge(left,right,on=['date','fecha','hora'], how='left'), dfs)


# Final dataset

In [ ]:
df_workspace.describe()

In [ ]:
df_workspace.columns

In [ ]:
def plot_resample_date(df,start_date,end_date, cols, resample_by):
    df[start_date:end_date][cols].resample(resample_by).mean().plot(figsize=(15, 6))
    plt.show()

In [ ]:
# set date as index
df_workspace.set_index('date', inplace=True)

# Review features by different resampling 

In [ ]:
start_year = str(2012)
end_year   = str(2020)

for s in ['W', 'MS', 'Q', 'Y']:
    print("SAMPLE BY: ", s)
    resample_by = s
    plot_resample_date(df_workspace,start_year,end_year ,[ 'PA_st'], resample_by )
    plot_resample_date(df_workspace,start_year,end_year ,[ 'UVA_st','UVB_st'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'CO_st0','CO_st1', 'CO_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'NO_st0', 'NO_st1', 'NO_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'NO2_st0', 'NO2_st1','NO2_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'O3_st0', 'O3_st1','O3_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'PM10_st0', 'PM10_st1', 'PM10_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'PM25_st0', 'PM25_st1', 'PM25_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'PMCO_st0', 'PMCO_st1', 'PMCO_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'SO2_st0', 'SO2_st1','SO2_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'RH_st0', 'RH_st1', 'RH_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'TMP_st0', 'TMP_st1','TMP_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'WDR_st0', 'WDR_st1', 'WDR_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'WSP_st0', 'WSP_st1', 'WSP_st2'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'dew_point_st'], resample_by)
    plot_resample_date(df_workspace,start_year,end_year ,[ 'rain_st'], resample_by)

# Forecasting with ARIMA

Time series provide the opportunity to forecast future values. Based on previous values, time series can be used to forecast trends in economics, weather, and capacity planning, to name a few. The specific properties of time-series data mean that specialized statistical methods are usually required. In this case pollution features will be predicted.

There are three distinct integers (p, d, q) that are used to parametrize ARIMA models. Because of that, ARIMA models are denoted with the notation ARIMA(p, d, q). Together these three parameters account for seasonality, trend, and noise in datasets:

    p is the auto-regressive part of the model. It allows us to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to be warm tomorrow if it has been warm the past 3 days.
    d is the integrated part of the model. This includes terms in the model that incorporate the amount of differencing (i.e. the number of past time points to subtract from the current value) to apply to the time series. Intuitively, this would be similar to stating that it is likely to be same temperature tomorrow if the difference in temperature in the last three days has been very small.
    q is the moving average part of the model. This allows us to set the error of our model as a linear combination of the error values observed at previous time points in the past.

When dealing with seasonal effects, we make use of the seasonal ARIMA, which is denoted as ARIMA(p,d,q)(P,D,Q)s. Here, (p, d, q) are the non-seasonal parameters described above, while (P, D, Q) follow the same definition but are applied to the seasonal component of the time series. The term s is the periodicity of the time series (4 for quarterly periods, 12 for yearly periods, etc.).

The seasonal ARIMA method can appear daunting because of the multiple tuning parameters involved. In the next section, we will describe how to automate the process of identifying the optimal set of parameters for the seasonal ARIMA time series model.


# Analizing parameters for ARIMA PM10 and CO

In [ ]:
There are several features, we focus in only two of them PM10 and CO2. These were selected because they
show different behavior over time. PM10 varies and is very errant likewise PM2.5 and O3.
In the other hand CO shows a repetitive pattern that ww believe ARIMA will give a result that we can use as a baseline


In [ ]:
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf

#Check autocorrelation to have an idea of parameters
for s in ['H','D','W', 'MS', 'Q', 'Y']:
    print("SAMPLE BY: ", s)
    resampled = df_workspace['PM10_st0'].resample(s).mean()
    autocorrelation_plot(resampled)
    plt.show()


In [ ]:
for s in ['H','D','W', 'MS', 'Q', 'Y']:
    print("SAMPLE BY: ", s)
    resampled = df_workspace['PM10_st0'].resample(s).mean()
    plot_pacf(resampled, lags=7)

In [ ]:
# 

# Create train test dataframes

In [ ]:
train_arima      = df_workspace['2017-01-01':'2018-12-31']
test_arima       = df_workspace['2019-09-01':'2019-12-31']
validation_arima = df_workspace['2020-01-01':'2020-02-29']

train_hours      = len(train_arima)
test_hours       = len(test_arima)
validation_hours = len(train_arima)


print(train_arima.shape)
print(test_arima.shape)
print(validation_arima.shape)



# Parameter Selection for the ARIMA Time Series Model

We used 2 years of training (2017-01-01 to 2018-12-31) and one year of training (2019-09-01 to 2019-12-31).

WWe then evaluated train dataset and compared predicted values against the true values of year 2019 
(2557 total hours in 2019)  year using brute force (try some of all possible combinations of parameters) 
to get hyperparameters for both features. We defined the trend string to be 'n' meaning that ot has NO trend.

Result of hyperparameter selection and the RMSE for year 2019 is:
 
 
 
PM10 for station 1 
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] with RMSE  9.394 
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] with RMSE  9.414
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] with RMSE  9.413
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] with RMSE  9.414
 > Model[[(1, 0, 0), (1, 0, 0, 0), 'n']] with RMSE 10.574
 > Model[[(0, 1, 0), (1, 0, 0, 0), 'n']] with RMSE 10.897
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'n']] with RMSE 10.897
 > Model[[(1, 1, 0), (0, 0, 0, 0), 'n']] with RMSE 10.897
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] with RMSE 17.313
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] with RMSE 23.729
 < Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] with RMSE 23.729
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] with RMSE 40.769

CO for station 1
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] with RMSE 0.774
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] with RMSE 0.777
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] with RMSE 0.777 
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] with RMSE 0.777
 > Model[[(0, 1, 0), (1, 0, 0, 0), 'n']] with RMSE 0.787
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'n']] with RMSE 0.787
 > Model[[(1, 1, 0), (0, 0, 0, 0), 'n']] with RMSE 0.787
 > Model[[(1, 0, 0), (1, 0, 0, 0), 'n']] with RMSE 0.851
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] with RMSE 1.164
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] with RMSE 1.475
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] with RMSE 1.475
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] with RMSE 2.404

 

 
   
 
 
 As expected PM10 has worst RMSE 40.769  compared to CO
 
 We decide using **Model[[(0, 0, 0), (1, 0, 0, 0), 'n']]** for all features 

In [ ]:
pip install funcy

In [ ]:
pip install numba

In [ ]:
# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# one-step sarima forecast
def sarima_forecast(history, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# walk-forward validation for univariate data
def walk_forward_validation(train, test, cfg):
    predictions = list()
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error

# score a model, return None on failure
def score_model(train, test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    # one failure during model validation suggests an unstable config
    try:
        # never show warnings when grid searching, too noisy
        with catch_warnings():
            filterwarnings("ignore")
            result = walk_forward_validation(train, test, cfg)
    except:
        error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

# grid search configs
def grid_search(train, test, cfg_list):
    scores = None
    # execute configs in parallel
    executor = Parallel(n_jobs=cpu_count(),  backend='multiprocessing')
    tasks = (delayed(score_model)(train, test, cfg) for cfg in cfg_list)
    scores = executor(tasks)
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores


def sarima_configs(seasonal=[0]):
    models = list()
    models.append([(1, 0, 0), (1, 0, 0, 0), 'n'])
    models.append([(1, 0, 0), (1, 0, 1, 0), 'n'])
    models.append([(1, 0, 0), (1, 0, 2, 0), 'n'])
    models.append([(2, 0, 0), (1, 0, 0, 0), 'n'])
    models.append([(2, 0, 0), (1, 0, 1, 0), 'n'])
    models.append([(2, 0, 0), (1, 0, 2, 0), 'n'])
    
    return models

def sarima_configs(seasonal=[0]):
    models = list()
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n'] # there are no trend 'c','t','ct'
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models


In [ ]:
cpus = cpu_count()
print("available processors: ", cpus )

In [ ]:




# model configs
cfg_list = sarima_configs()
#Carbon monoxide , Nitrogen Dioxide, Sulfur dioxide
arima_models = ['PM10_st0','CO_st0']#'NO2_st0', 'SO2_st0']

for m in arima_models:
    # grid search
    print('Grid search for: ', m)
    scores = grid_search(train_arima[m],test_arima[m], cfg_list )
    # list top 3 configs
    for cfg, error in scores[:3]:
        print(cfg, error)  
        print('done for: ', m) 
